# CTA Data Analysis

### ctapipe
This tutorial uses ctapipe, a framework for prototyping the low-level data processing algorithms for the Cherenkov Telescope Array. To get more information about the instalation and tutorials of ctapipe go to: https://cta-observatory.github.io/ctapipe/index.html

The public files here described are available at https://doi.org/10.5281/zenodo.7298569.

### Worflow: <br>
1. Open file and explore the dataset
>- See internal structure and metadata in HDF5 file.
>- Usage of ctapipe TableLoader
>- Subarray description
>- DL1: DL1a and DL1b. Plot images and see the parameters

## 1. Open file and explore the dataset.

### See internal structure and metadata in HDF5 file:

In [1]:
# Load necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u

# Packages to explore HDF5 file
import tables
from ctapipe.io import read_table

# Packages for subarray description
from ctapipe.instrument import SubarrayDescription
from ctapipe.visualization import CameraDisplay

# Packages for extracting the images parameters
from ctapipe.image import hillas_parameters
#from ctapipe.image import timing_parameters
#from ctapipe.image import number_of_islands
#from ctapipe.image import camera_to_shower_coordinates


In [5]:
hdf_file = "gamma-diffuse_with_images_00.dl2.h5"
h5file = tables.open_file(hdf_file, mode="r", title="DL file")

*See metadata related to the moment when the gamma file was created or modified:*

In [ ]:
h5file.root._v_attrs

*HDF5 files internal structure is composed of tables inside containers that we can browse at different leves:*

In [ ]:
h5file

To visualize any of the tables, for instance /simulation/service/shower_distribution:

In [3]:
shower_distribution = read_table(hdf_file, '/simulation/service/shower_distribution')
shower_distribution[0:5]

OSError: ``/Users/nalvarez/Documents/ESCAPE/cta/dataset/zenodo_gamma-diffuse/gamma-diffuse_with_images_00.dl2.h5`` does not exist

### * Tip
To explore a HDF5 file through a GUI, you can use [vitables](https://vitables.org/) or [HDFView](https://www.hdfgroup.org/downloads/hdfview/). 

### ctapipe.io.TableLoader
ctpipe offers several ways to access the tables. Another convenient tool is to load the data using **ctapipe.io.TableLoader**

See the [TableLoader docs](https://cta-observatory.github.io/ctapipe/api/ctapipe.io.TableLoader.html#ctapipe.io.TableLoader)

In [ ]:
from ctapipe.io import TableLoader

In [ ]:
opts = dict(load_dl2=True, load_dl1_images=False, load_simulated=True)
with TableLoader("/Users/nalvarez/Documents/ESCAPE/cta/dataset/zenodo_gamma-diffuse/gamma-diffuse_with_images_00.dl2.h5", **opts) as loader:
    subarray = loader.subarray
    gamma_observations = loader.read_observation_information() 
    gamma_events = loader.read_subarray_events() 
    gamma_tel_events = loader.read_telescope_events() 
    gamma_hists = loader.read_shower_distribution() 
    gamma_sim_configs = loader.read_simulation_configuration() 

In [ ]:
gamma_observations[0:5]

In [ ]:
gamma_events[0:5]

Take into account that TableLoader loads different hdf5 file tables into the same output table, in order to have all the relevant parameters for a study. For instance:
- **gamma_observations** is the table /configuration/observation/observation_block 
- **gamma_events** is a combination of the tables dl1/event/subarray/trigger, simulation/event/subarray/shower and dl1/event/subarray/geometry/HillasReconstructor
- **gamma_tel_events** is a combination of the tables dl1/event/telescope/parameters_tel_, dl2/event/telescope/impact/HillasReconstructor/tel_, simulation/event/telescope/impact/tel_, dl1/event/subarray/trigger, simulation/event/subarray/shower and dl1/event/subarray/geometry/HillasReconstructor
- **gamma_hists** is the table /simulation/service/shower_distribution
- **gamma_sim_configs** is the table /configuration/simulation/run

*For instance:*

In [ ]:
gamma_hists[0:5], read_table(hdf_file, '/simulation/service/shower_distribution')[0:5]

In [ ]:
def info(obs, events, tel_events):
    n_runs = len(obs)
    n_events = len(events)
    n_valid_stereo = np.count_nonzero(events['HillasReconstructor_is_valid'])
    n_tel_events = len(tel_events)
    n_valid_params = np.count_nonzero(tel_events['hillas_width'] > 0)
    
    print(f"Runs: {n_runs: 8d}")
    print(f"Events: {n_events: 8d}")
    print(f"With valid stereo: {n_valid_stereo: 8d}")
    print(f"Tel-Events: {n_tel_events: 8d}")
    print(f"With valid params: {n_valid_params: 8d}")

In [ ]:
print("Gammas")
info(gamma_observations, gamma_events, gamma_tel_events)

- **Event:** a basic unit of data, detection associated with a cascade of particles. There are also calibration events, created via laser or pedestal, to estimate the noise.
- **Run:** set of events taken in similar conditions, i.e., pointing to the same astronomical source in a maximum time of 20 min (since the instrument response depends of zenital and azimutal angles).


In [ ]:
fig, ax = plt.subplots(layout="constrained")
multiplicity = np.count_nonzero(gamma_events['HillasReconstructor_telescopes'],axis=1)
bin_centers = np.arange(len(subarray) + 1)
bins = np.append(bin_centers - 0.5, bin_centers[-1] + 0.5)
ax.hist(
    multiplicity,
    bins=bins,
    linewidth=2,
    edgecolor='w',
)
ax.set_xlabel('Multiplicity')
ax.set_xticks(bin_centers)
None

### Subarray description
Here we will explore the telescope setup.

*Read instrument description, needed for camera display:*

In [ ]:
subarray = SubarrayDescription.from_hdf(hdf_file)
subarray

In [ ]:
subarray.info()

In [ ]:
subarray.peek()

In [ ]:
subarray.to_table()

In [ ]:
subarray.tel

*Get a table of the optics description:*

In [ ]:
subarray.to_table(kind="optics")

*Choose one telescope to display properties, for instance LST, tel_id 1:*

In [ ]:
tel = subarray.tel[1]
tel

In [ ]:
tel.optics

In [ ]:
tel.camera.geometry

In [ ]:
tel.camera.geometry.to_table()

*Visualize the camera:*

In [ ]:
disp = CameraDisplay(tel.camera.geometry)

### DL1

DL1 is split in two sub-levels

* __DL1a__: calibrated images in units of photoelectrons (p.e.) and information about the time of arrival of the signal to each pixel.
* __DL1b__: parametrizations of the DL1a images, the so-called "Hillas parameters".

### DL1a: Images table:
*See images table for the LSTCam tel_id 001:*

In [ ]:
dl1_images = read_table(hdf_file, '/dl1/event/telescope/images/tel_001')
dl1_images

The images table stores, for each event and pixel:

- the charge in photo electrons ("image")
- the time of the light arrival in ns ("peak time")
- whether the pixel survived cleaning or not ("image_mask"). 

*There are 5987 different events, here we choose as an example the number 1:*

In [ ]:
event = 2198
dl1_singleim = np.array(dl1_images['image'][event])
dl1_singlepeak = np.array(dl1_images['peak_time'][event])
obs_id = dl1_images['obs_id'][event]
event_id = dl1_images['event_id'][event]
print("obs_id:",obs_id,"event_id:",event_id)

*Show the image:*

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
d1 = CameraDisplay(tel.camera.geometry, image=dl1_singleim, ax=ax1)
d2 = CameraDisplay(tel.camera.geometry, image=dl1_singlepeak, ax=ax2)

ax1.set_title("Image (p.e.)")
d1.add_colorbar(ax=ax1)

ax2.set_title("Peak time (ns)")
d2.add_colorbar(ax=ax2)


*Remove pixels that did not survived cleaning using "image_mask":*

In [ ]:
dl1_mask = np.array(dl1_images['image_mask'])
dl1_singlemask = dl1_mask[event]

In [ ]:
cleaned = dl1_singleim.copy()
cleaned[~dl1_singlemask] = 0 

In [ ]:
cleaned_peak = dl1_singlepeak.copy()
cleaned_peak[~dl1_singlemask] = 0 

*Remove the image pixels that did not passed the clean cut:*

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
d1 = CameraDisplay(tel.camera.geometry, image=cleaned, ax=ax1)
d2 = CameraDisplay(tel.camera.geometry, image=cleaned_peak, ax=ax2)

ax1.set_title("Image (p.e.)")
d1.add_colorbar(ax=ax1)

ax2.set_title("Peak time (ns)")
d2.add_colorbar(ax=ax2)

*Now highlight the image pixels that passed the clean cut:*

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

d1 = CameraDisplay(tel.camera.geometry, image=dl1_singleim, ax=ax1)
d2 = CameraDisplay(tel.camera.geometry, image=dl1_singlepeak - np.average(dl1_singlepeak, weights=dl1_singlepeak), ax=ax2)

ax1.set_title("Image (p.e.)")
d1.add_colorbar(ax=ax1)
d1.highlight_pixels(dl1_singlemask, color="red", linewidth=1)

ax2.set_title("Pulse Time (ns)")
d2.add_colorbar(ax=ax2)

### DL1b: Image parameters:
See parameters table for the LSTCam:

In [ ]:
dl1_parameters = read_table(hdf_file, '/dl1/event/telescope/parameters/tel_001')
dl1_parameters

There are 3 types of parameters that describe the shower: 
- Hillas parameters.
- Leakage parameters.
- Concentration parameters.

### Hillas parameters:
The Hillas parameters are a set of geometric features that describe the orientation and size of an ellipse fitted to the area of a shower image, relying
on the fact that the gamma-ray images in the camera focal plane are, to a good approximation, elliptical in shape. These parameters are:
- image centroid or "center of gravity" (x, y)
- length and width of the ellipse
- size (total image amplitude)
- nominal distance d (angular distance between the centre of the camera and the image centre of gravity)
- azimuthal angle of the image main axis φ
- orientation angle α 

All parameters are calculated using the charge, time and coordinates of the pixels surviving the cleaning. The Hillas parameters can be used to estimate event properties, e.g. using random forests. 


![sketch_with_disp_angle.png](sketch_with_disp_angle.png)
Image adapted from https://github.com/cta-observatory/ctapipe/issues/1078#issuecomment-495663160


*It is possible to extract the Hillas parameters directly from DL1a images:*

In [ ]:
hillas = hillas_parameters(tel.camera.geometry, cleaned)
print(hillas)
hillas

*Plot Hillas parameters over the cleaned image:*

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(9, 6))

disp = CameraDisplay(tel.camera.geometry, image=cleaned, title = "Image (p.e.)")
disp.add_colorbar()
disp.overlay_moments(hillas, color="yellow", lw = 2)

plt.xlim(hillas.x.to_value(u.m) - 0.5, hillas.x.to_value(u.m) + 0.5)
plt.ylim(hillas.y.to_value(u.m) - 0.5, hillas.y.to_value(u.m) + 0.5)

*Hillas parameters from the given table:*

In [ ]:
dl1_parameters[0:5]

*Choose the parameters that correspond to the obs_id and event_id of the selected image:*

In [ ]:
hillas_singleim = dl1_parameters[(dl1_parameters['obs_id'] == obs_id) & (dl1_parameters['event_id'] == event_id)
                                & (dl1_parameters['tel_id'] == 1)]

In [ ]:
hillas_singleim

### ctapipe.io.TableLoader
Now we will load all four LST telescopes and plot their images using **ctapipe.io.TableLoader**

In [ ]:
with TableLoader(
    "/Users/nalvarez/Documents/ESCAPE/cta/dataset/zenodo_gamma-diffuse/gamma-diffuse_with_images_00.dl2.h5",
    load_dl1_images=True,
    load_true_images=True,
    load_simulated=True,
) as loader: 
    lst_images = loader.read_telescope_events("LST_LST_LSTCam", start=0,stop=10000)

In [ ]:
lst_images[0:5]

In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension


In [ ]:
from ipywidgets import interact, IntSlider
from ctapipe.coordinates import TelescopeFrame
camera = subarray.tel[1].camera.geometry.transform_to(TelescopeFrame())
fig, (ax_image, ax_time) = plt.subplots(1, 2, layout="constrained", figsize=(9,4.5))
d_image = CameraDisplay(
    camera,
    image=lst_images['image'][0],
    ax=ax_image
)
d_image.add_colorbar()
d_time = CameraDisplay(
    camera,
    image=lst_images['peak_time'][0],
    ax=ax_time,
    cmap=plt.get_cmap('RdBu_r').with_extremes(bad='gray')
)
d_time.add_colorbar()

def update(index):
    row = lst_images[index]
    d_image.image = row['image']
    d_time.image = np.where(row['image_mask'], row['peak_time'], np.nan)
    ax_image.set_title(f'obs_id {row["obs_id"]}, event_id {row["event_id"]},tel_id {row["tel_id"]}')
    ax_time.set_title(f'true_energy {row["true_energy"]:.3f} TeV')
    
interact(update, index=IntSlider(min=0, max=len(lst_images) - 1, value=1900))